In [31]:
import pickle
jpg_path = '/content/drive/MyDrive/CS230Project/SSDD/JPEGImages_sub/'
root = '/content/drive/MyDrive/CS230Project/SSDD/'
with open(root + "standardDict.pkl", "rb") as input_file:
  img = pickle.load(input_file)

In [103]:
import os
xml_path = '/content/drive/MyDrive/CS230Project/SSDD/Annotations_sub/'
anns = os.listdir(xml_path)
labels_df = pd.read_csv(root + 'labels.csv')
grouped = labels_df.groupby('filename')

In [104]:
dest = '/content/drive/MyDrive/CS230Project/SSDD/datasets/'
test['type'] = dest + 'test/'
train['type'] = dest + 'train/'
test_offshore['type'] = dest + 'test_offshore/'
test_inshore['type'] = dest + 'test_inshore/'

In [122]:
def standardize(path, annotations, df, grouped):
  anns_id = [i.split('.')[0] for i  in annotations]
  anns_df = pd.DataFrame(anns_id, columns=['file'])
  # Attach correct image file path
  anns_df = df.merge(df, 
                     on = 'file',  
                     how = 'inner')
  # Give default names
  std = {i : {"file_name": df.loc[df['file'] == i]['type'].values[0] + i + '.jpg',
              "height":800,
              "width":800,
              "image_id" : i,
              "annotations":[{}]} for i in df['file'].values}
  # Overwrite files with annotations
  keys = grouped.groups.keys()
  for i in keys:
    if i.split('.')[0] in std:
      j = grouped.get_group(i)
      temp = []
      for index, row in j.iterrows():
        ann_temp = {}
        ann_temp['bbox'] = [row['xmin'],  
                            row['ymin'],  
                            row['xmax'],  
                            row['ymax']]
        ann_temp['bbox_mode'] = 0
        ann_temp['category_id'] = 0
        temp.append(ann_temp)
      std[i.split('.')[0]]['annotations'] = temp
  # Cache
  f = open(path + "standardDict.pkl","wb")
  pickle.dump(list(std.values()),f)
  f.close()
  return std

In [125]:
a = standardize(dest + 'train/', anns, train, grouped)

In [126]:
len(a)

6000